In [ ]:
#import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
d = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
d.head()

In [ ]:
testd = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
testd.head()

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
d_encoded = pd.get_dummies(d, columns = ['country' , 'store' , 'product']) 
d_encoded.head()

In [ ]:
d=d_encoded
testd=pd.get_dummies(testd, columns = ['country' , 'store' , 'product']) 

In [ ]:
d['date']=pd.to_datetime(d['date'])
testd['date']=pd.to_datetime(testd['date'])

In [ ]:
d['day']=d['date'].dt.day
d['month']=d['date'].dt.month
d['year']=d['date'].dt.year
testd['day']=testd['date'].dt.day
testd['month']=testd['date'].dt.month
testd['year']=testd['date'].dt.year

In [ ]:
davg=d.groupby('date')['num_sold'].mean()
davg.plot()

In [ ]:
davg=pd.DataFrame(davg)
davg.head()

In [ ]:
dt=pd.DataFrame(testd.groupby('date')['country_Norway'].mean())
dt.columns=['num_sold']
dt.head()

In [ ]:
dtall=pd.concat([davg,dt],axis=0)

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(freq="M", order=4)  # 10 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=dtall.index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample()  # create features for dates in tunnel.index

In [ ]:
Xtimetrain=X.loc[:'2018-12-31']
Xtimetest=X.loc['2019-01-01':]

In [ ]:
d.index=d['date']
testd.index=testd['date']
d.head()

In [ ]:
d=d.join(Xtimetrain)
testd=testd.join(Xtimetest)

In [ ]:
testd.head()

In [ ]:
d=d.drop('date',axis=1)
testd=testd.drop('date',axis=1)

In [ ]:
d=d.drop('row_id',axis=1)

In [ ]:
X=d.drop('num_sold',axis=1)
y=d['num_sold']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()
cat.fit(X_train,y_train)

In [ ]:
pv=cat.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(pv,y_valid))

In [ ]:
s = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
s.head()

In [ ]:
testd.head()

In [ ]:
X_test=testd.drop('row_id',axis=1)
pvtest=cat.predict(X_test)

In [ ]:
w=pd.DataFrame()
w['row_id']=testd['row_id']

In [ ]:
w.head()   

In [ ]:
w['num_sold']=w['row_id']

In [ ]:
for i in range(w.shape[0]):
    w['num_sold'][i]=pvtest[i]

In [ ]:
w.head()

In [ ]:
w.to_csv('submission.csv', index=False)   